## Initialization settings - Load the packages

Load the required packages and define the function to fix the random seed

In [ ]:
# The initialization settings - Load the required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
import tensorflow.keras as tkeras
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import DBSCAN
from statsmodels.tsa.seasonal import seasonal_decompose
import random

# Note that 'edward2' is not a built-in Python package of Kaggle.
# When you first load it, you need to install it using 'pip'.
import os
os.system('pip install edward2')
import edward2 as ed2

# Define the function to fix the random seed
def set_seed(seed = 0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

## Part 1 - The exploration on the Wine data set

Load the Wine data set  
You first need to upload this data set to the following path on Kaggle:  
**'../input/wine-dataset/wine.csv'**  

Perform some initial analyses on Wine data set

In [ ]:
# Load the Wine data set
# You first need to upload this data set to the following path:
# '../input/wine-dataset/wine.csv'
wine = pd.read_csv('../input/wine-dataset/wine.csv', sep = ',')
dataset_size = len(wine)

# Fix the random seed
set_seed(seed = 60)

# Some initial analyses on Wine data set
print(wine.info()); display(wine.describe())
plt.figure(figsize = (12, 8))
sns.heatmap(wine.corr(), annot = True, cmap = 'seismic',
            vmin = -1, vmax = 1, center = 0)
plt.title('Correlation matrix for Wine data set')
display(wine[['quality']].join(pd.DataFrame({'count' : [1 for i in range(dataset_size)]})).\
    groupby('quality').count())

Visualize the Wine data set on 2-dimensional space using the Principal Component Analysis (PCA) technique

In [ ]:
# In order to visualize the Wine data set, I use Principal Component Analysis
# (PCA) technique to reduce the dimension of the features of this data set.
model_dimredu = PCA(n_components = 2)
wine_features = wine.drop(['quality'], 1)
model_dimredu.fit(wine_features)
wine_features_2D = model_dimredu.transform(wine_features)

# I visualize the Wine data set on 2-dimensional space, with the
# data points classified by wine quality.
wine_dimredu = pd.concat(
    [pd.DataFrame(wine_features_2D, columns = ['PCA1', 'PCA2']),\
    wine[['quality']].rename(columns = {'quality' : 'Quality'})], axis = 1)
sns.lmplot(x = 'PCA1', y = 'PCA2', hue = 'Quality', data = wine_dimredu, fit_reg = False)
plt.title('Visualization of Wine data set on 2-dimensional space')

Prepare the data used to train the model and split them into train, validation and test set

In [ ]:
# Prepare the data used to train the model
# Prepare the features and target of the Wine data set separately, convert the
# data set to "tensorflow.dataset" type and convert "wine quality" to "float" type
features = tf.constant(wine.drop(['quality'], 1))
labels = tf.constant(wine[['quality']])
wine_tfds = tf.data.Dataset.from_tensor_slices((features, labels)).\
    map(lambda x, y: (x, tf.cast(y, tf.float64))).\
    shuffle(buffer_size = dataset_size).prefetch(buffer_size = dataset_size)

# Split the data set into train, validation and test set, and batch each set
train_size = round(dataset_size*0.8)
validation_size = round(dataset_size*0.1)
test_size = dataset_size - train_size - validation_size
batch_size = 256
wine_train = wine_tfds.take(train_size).batch(batch_size)
wine_validation = wine_tfds.skip(train_size).take(validation_size).batch(validation_size)
wine_test = wine_tfds.skip(train_size + validation_size).batch(test_size)

Define the prior and variational posterior of network weight parameters and define the negative log-likelihood function of the model

In [ ]:
# Define the prior weight distributions as independent standard normal distributions
def prior(kernel_size, bias_size, dtype = None):
    n = kernel_size + bias_size
    return tkeras.Sequential([
        tfp.layers.DistributionLambda(
            lambda t: tfp.distributions.MultivariateNormalDiag(
                loc = tf.zeros(n), scale_diag = tf.ones(n))
        )
    ])

# Define the variational posterior weight distribution as multivariate Gaussian distribution
# The trainable parameters for this distribution are the means, variances, and covariances.
def posterior(kernel_size, bias_size, dtype = None):
    n = kernel_size + bias_size
    return tkeras.Sequential([
               tfp.layers.VariableLayer(
                   tfp.layers.MultivariateNormalTriL.params_size(n), dtype = dtype
               ),
               tfp.layers.MultivariateNormalTriL(n),
           ])

# Define the negative log-likelihood function of the model
def negative_log_likelihood(y_true, y_pred):
    return -y_pred.log_prob(y_true)

Construct and train the Bayesian neural network (BNN) model

In [ ]:
# Specify some model parameters
kl_loss_weight = 1/train_size
learning_rate = 0.001
num_epochs = 1000

# Construct the Bayesian neural network model with two hidden layers
wine_model = tkeras.Sequential([
    tkeras.layers.Input(shape = (12,)),
    tkeras.layers.BatchNormalization(),
    tfp.layers.DenseVariational(
        units = 8, make_posterior_fn = posterior,
        make_prior_fn = prior, kl_weight = kl_loss_weight,
        activation = 'sigmoid'
    ),
    tfp.layers.DenseVariational(
        units = 8, make_posterior_fn = posterior,
        make_prior_fn = prior, kl_weight = kl_loss_weight,
        activation = 'sigmoid'
    ),
    tkeras.layers.Dense(units = 2),
    tfp.layers.IndependentNormal(1)
])

# View the structure of the model
wine_model.summary()

# Compile the constructed Bayesian neural network
# We take the negative Evidence Lower Bound (-ELBO) as the loss function,
# and use the RMSprop optimizer with learning rate being equal to 0.001 to
# minimize the loss function, and use Mean Square Error (MSE) as the metric
# to evaluate the accuracy of the model.
wine_model.compile(
    optimizer = tkeras.optimizers.RMSprop(learning_rate = learning_rate),
    loss = negative_log_likelihood,
    metrics = [tkeras.metrics.mean_squared_error]
)

# Fit the constructed Bayesian neural network with data
wine_fit = wine_model.fit(x = wine_train, epochs = num_epochs,
                          validation_data = wine_validation)

Draw the trend of the loss and the MSE on the train and validation set during the training process respectively

In [ ]:
# Draw the trend of the loss and the MSE on the train and
# validation set during the training process respectively

# Prepare the data
train_loss = wine_fit.history['loss']
val_loss = wine_fit.history['val_loss']
train_eva = wine_fit.history['mean_squared_error']
val_eva = wine_fit.history['val_mean_squared_error']
epochs = range(1, num_epochs + 1)

# The trend of loss on the train and validation set
fig1 = plt.figure(figsize = (5, 3)); ax1 = plt.axes()
ax1.plot(epochs, train_loss, 'b-.', label = 'Training loss')
ax1.plot(epochs, val_loss, 'g-.', label = 'Validation loss')
ax1.xaxis.set_major_locator(plt.MultipleLocator(100))
ax1.set(xlabel = 'Epoch', ylabel = 'Loss',
        title = 'Training and validation loss')
ax1.legend()

# The trend of the metric (MSE) on the train and validation set
fig2 = plt.figure(figsize = (5, 3)); ax2 = plt.axes()
ax2.plot(epochs, val_eva, 'g-', label = 'Validation MSE')
ax2.plot(epochs, train_eva, 'b-', label = 'Training MSE')
ax2.xaxis.set_major_locator(plt.MultipleLocator(100))
ax2.set(xlabel = 'Epoch', ylabel = 'Mean squared error',
        title = 'Training and validation mean squared error')
ax2.legend()

# Evaluate the trained model on both train set and test set respectively
print(wine_model.evaluate(wine_train, verbose = 0))
print(wine_model.evaluate(wine_test, verbose = 0))

I take 12 samples from the test set, two samples per quality level (from 3 to 8). Then, I construct and visualize the 95% confidence intervals for the predictions of these selected samples.

In [ ]:
# Take 12 samples from the test set, two samples per quality level (from 3 to 8)
features_exa, targets_exa = list(wine_test)[0]
features_exa = features_exa.numpy()
targets_exa = targets_exa.numpy()[:, 0]
sample_ind = np.array([])
for k in range(3, 9):
    choice = np.random.choice(np.where(targets_exa == k)[0], size = 2)
    sample_ind = np.concatenate([sample_ind, choice])
examples = features_exa[sample_ind.astype('int'), :]
labels_exa = targets_exa[sample_ind.astype('int')]

# Compare the prediction means with the true labels
examples_mean = wine_model(examples).mean().numpy()
examples_std = wine_model(examples).stddev().numpy()

# Construct and visualize the 95% confidence intervals for the predictions
plt.figure(figsize = (10, 5))
index = range(1, 13)
quality = np.concatenate(np.array([[k, k] for k in range(3, 9)]))
plt.scatter(index, examples_mean, color = 'red', label = 'Prediction mean')
plt.scatter(index, labels_exa, color = 'green', label = 'True value')
plt.scatter(index, examples_mean + 1.96*examples_std,
            color = 'blue', marker = '_', label = 'Confidence bound')
plt.scatter(index, examples_mean - 1.96*examples_std,
            color = 'blue', marker = '_')
plt.xlabel('True wine quality'); plt.ylabel('Wine quality')
plt.xticks(ticks = index, labels = quality)
plt.title('The 95% confidence intervals for the predictions of twelve selected samples')
plt.legend()

Clear the model and re-train the model with the whole data set

In [ ]:
# Clear the model and re-train the model with the whole data set

# Update the weight for the KL divergence loss between
# the surrogate posterior and weight prior
kl_loss_weight = 1/dataset_size

# Clear the model
wine_model = tkeras.Sequential([
    tkeras.layers.Input(shape = (12,)),
    tkeras.layers.BatchNormalization(),
    tfp.layers.DenseVariational(
        units = 8, make_posterior_fn = posterior,
        make_prior_fn = prior, kl_weight = kl_loss_weight,
        activation = 'sigmoid'
    ),
    tfp.layers.DenseVariational(
        units = 8, make_posterior_fn = posterior,
        make_prior_fn = prior, kl_weight = kl_loss_weight,
        activation = 'sigmoid'
    ),
    tkeras.layers.Dense(units = 2),
    tfp.layers.IndependentNormal(1)
])

# Re-compile the model with the same settings
wine_model.compile(
    optimizer = tkeras.optimizers.RMSprop(learning_rate = learning_rate),
    loss = negative_log_likelihood,
    metrics = [tkeras.metrics.mean_squared_error]
)

# Re-fit the model with the whole data set
wine_model.fit(x = wine_tfds.batch(batch_size), epochs = num_epochs)

Quantify and plot all kinds of uncertainties of the predictions

In [ ]:
# Quantify all kinds of uncertainties of the predictions
N = 1000; records = np.zeros((N, dataset_size, 3))
for i in range(N):
    records[i, :, 0] = wine_model(features).mean().numpy()[:, 0]
    records[i, :, 1] = wine_model(features).variance().numpy()[:, 0]
    records[i, :, 2] = ((wine_model(features).mean().numpy() - labels.numpy())**2)[:, 0]
epistemic = np.var(records[:, :, 0], axis = 0)
aleatoric = np.mean(records[:, :, 1], axis = 0)
misspecification = np.mean(records[:, :, 2], axis = 0)
P = 1; Q = 1; R = 1
comp_uncer = P*epistemic + Q*aleatoric + R*misspecification

# Plot all kinds of uncertainties

# Plot the distribution of the epistemic uncertainty
plt.figure(figsize = (5, 3))
plt.hist(epistemic, color = 'blue', bins = 25,
         weights = np.ones(dataset_size)/dataset_size)
plt.xlabel('Epistemic uncertainty')
plt.ylabel('Frenquency')
plt.title('The distribution of the epistemic uncertainty')

# Plot the distribution of the aleatoric uncertainty
plt.figure(figsize = (5, 3))
plt.hist(aleatoric, color = 'green', bins = 25,
         weights = np.ones(dataset_size)/dataset_size)
plt.xlabel('Aleatoric uncertainty')
plt.ylabel('Frenquency')
plt.title('The distribution of the aleatoric uncertainty')

# Plot the distribution of the model misspecification uncertainty
plt.figure(figsize = (5, 3))
plt.hist(misspecification, color = 'red', bins = 25,
         weights = np.ones(dataset_size)/dataset_size)
plt.xlabel('Misspecification uncertainty')
plt.ylabel('Frenquency')
plt.title('The distribution of the misspecification uncertainty')

# Plot the distribution of the total prediction uncertainty
plt.figure(figsize = (5, 3))
plt.hist(comp_uncer, color = 'purple', bins = 25,
         weights = np.ones(dataset_size)/dataset_size)
plt.xlabel('Total prediction uncertainty')
plt.ylabel('Frenquency')
plt.title('The distribution of the total prediction uncertainty')
threshold = 3
plt.axvline(threshold, color = 'red')

# Record the information about the index of outliers
index = (comp_uncer > threshold).astype('int').tolist()
wine_BNN = wine_dimredu.join(pd.DataFrame(index, columns = ['indicator'])).\
                        join(pd.DataFrame(comp_uncer, columns = ['uncertainty']))
print(np.mean(index)); print(np.sum(index))

Visualize the observations with high epistemic, aleatoric and model misspecification uncertainty respectievly

In [ ]:
# Determine a threshold about the uncertainty level
quantile = 0.95

# Visualize the observations with high epistemic uncertainty
plt.figure(figsize = (5, 5))
ax1 = plt.axes(projection = '3d')
epi_index = (epistemic >= np.quantile(epistemic, quantile))
ax1.scatter3D(wine_BNN['PCA1'][np.where(1 - epi_index)[0]],
              wine_BNN['PCA2'][np.where(1 - epi_index)[0]],
              wine_BNN['Quality'][np.where(1 - epi_index)[0]],
              c = 'gray', s = 10, alpha = 0.25)
ax1.scatter3D(wine_BNN['PCA1'][np.where(epi_index)[0]],
              wine_BNN['PCA2'][np.where(epi_index)[0]],
              wine_BNN['Quality'][np.where(epi_index)[0]],
              c = 'red', s = 10 + 200*epistemic[np.where(epi_index)[0]])
ax1.scatter3D([], [], [], c = 'gray', s = 10, label = 'Normal')
ax1.scatter3D([], [], [], c = 'red', s = 10, label = 'High uncertainty')
ax1.set(xlabel = 'PCA1', ylabel = 'PCA2', zlabel = 'Wine quality',
        title = 'High epistemic uncertainty samples')
plt.legend()

# Visualize the observations with high aleatoric uncertainty
plt.figure(figsize = (5, 5))
ax2 = plt.axes(projection = '3d')
alea_index = (aleatoric >= np.quantile(aleatoric, quantile))
ax2.scatter3D(wine_BNN['PCA1'][np.where(1 - alea_index)[0]],
              wine_BNN['PCA2'][np.where(1 - alea_index)[0]],
              wine_BNN['Quality'][np.where(1 - alea_index)[0]],
              c = 'gray', s = 10, alpha = 0.25)
ax2.scatter3D(wine_BNN['PCA1'][np.where(alea_index)[0]],
              wine_BNN['PCA2'][np.where(alea_index)[0]],
              wine_BNN['Quality'][np.where(alea_index)[0]],
              c = 'red', s = 10 + 10*aleatoric[np.where(alea_index)[0]])
ax2.scatter3D([], [], [], c = 'gray', s = 10, label = 'Normal')
ax2.scatter3D([], [], [], c = 'red', s = 10, label = 'High uncertainty')
ax2.set(xlabel = 'PCA1', ylabel = 'PCA2', zlabel = 'Wine quality',
        title = 'High aleatoric uncertainty samples')
plt.legend()

# Visualize the observations with high misspecification uncertainty
plt.figure(figsize = (5, 5))
ax3 = plt.axes(projection = '3d')
mis_index = (misspecification >= np.quantile(misspecification, quantile))
ax3.scatter3D(wine_BNN['PCA1'][np.where(1 - mis_index)[0]],
              wine_BNN['PCA2'][np.where(1 - mis_index)[0]],
              wine_BNN['Quality'][np.where(1 - mis_index)[0]],
              c = 'gray', s = 10, alpha = 0.25)
ax3.scatter3D(wine_BNN['PCA1'][np.where(mis_index)[0]],
              wine_BNN['PCA2'][np.where(mis_index)[0]],
              wine_BNN['Quality'][np.where(mis_index)[0]],
              c = 'red', s = 10 + 1.6*misspecification[np.where(mis_index)[0]])
ax3.scatter3D([], [], [], c = 'gray', s = 10, label = 'Normal')
ax3.scatter3D([], [], [], c = 'red', s = 10, label = 'High uncertainty')
ax3.set(xlabel = 'PCA1', ylabel = 'PCA2', zlabel = 'Wine quality',
        title = 'High misspecification uncertainty samples')
plt.legend()

Perform the Local Outlier Factor (LOF) method and DBSCAN method on Wine data set

In [ ]:
# Perform the Local Outlier Factor (LOF) method on Wine data set
Lof = LocalOutlierFactor()
Lof_pre = Lof.fit_predict(wine)
Lof_fac = Lof.negative_outlier_factor_
wine_LOF = wine_dimredu.join(pd.DataFrame(Lof_pre, columns = ['indicator'])).\
                        join(pd.DataFrame(Lof_fac, columns = ['factor']))

# Perform the DBSCAN method on Wine data set
DBSCAN_model = DBSCAN(eps = 8, min_samples = 5)
DBSCAN_fit = DBSCAN_model.fit(wine)
DBSCAN_pre = DBSCAN_fit.labels_
DBSCAN_noise = (DBSCAN_pre == -1).astype('int')
wine_DBSCAN = wine_dimredu.join(pd.DataFrame(DBSCAN_noise, columns = ['indicator']))

Visualize the results of outlier detection using BNN, LOF and DBSCAN method respectively

In [ ]:
# Visualize the effects of the three outlier detection methods on Wine data set

# The visualization of the effect of Bayesian neural network
plt.figure(figsize = (5, 5))
ax1 = plt.axes(projection = '3d')
uncer_outlier = wine_BNN['uncertainty'][wine_BNN['indicator'] == 1]
ax1.scatter3D(wine_BNN['PCA1'][wine_BNN['indicator'] == 0],
              wine_BNN['PCA2'][wine_BNN['indicator'] == 0],
              wine_BNN['Quality'][wine_BNN['indicator'] == 0],
              c = 'gray', s = 10, alpha = 0.25)
ax1.scatter3D(wine_BNN['PCA1'][wine_BNN['indicator'] == 1],
              wine_BNN['PCA2'][wine_BNN['indicator'] == 1],
              wine_BNN['Quality'][wine_BNN['indicator'] == 1],
              c = 'red', s = 10 + 2.5*uncer_outlier)
ax1.scatter3D([], [], [], c = 'gray', s = 10, label = 'Inlier')
ax1.scatter3D([], [], [], c = 'red', s = 10, label = 'Outlier')
ax1.set(xlabel = 'PCA1', ylabel = 'PCA2', zlabel = 'Wine quality',
        title = 'Outlier detection on Wine data set\nusing Bayesian neural network')
plt.legend()

# The visualization of the effect of Local Outlier Factor method
plt.figure(figsize = (5, 5))
ax2 = plt.axes(projection = '3d')
ax2.scatter3D(wine_LOF['PCA1'][wine_LOF['indicator'] == 1],
              wine_LOF['PCA2'][wine_LOF['indicator'] == 1],
              wine_LOF['Quality'][wine_LOF['indicator'] == 1],
              c = 'gray', s = 10, alpha = 0.25)
LOF_outlier = wine_LOF['factor'][wine_LOF['indicator'] == -1]
ax2.scatter3D(wine_LOF['PCA1'][wine_LOF['indicator'] == -1],
              wine_LOF['PCA2'][wine_LOF['indicator'] == -1],
              wine_LOF['Quality'][wine_LOF['indicator'] == -1],
              c = 'red', s = 10 - 4*LOF_outlier)
ax2.scatter3D([], [], [], c = 'gray', s = 10, label = 'Inlier')
ax2.scatter3D([], [], [], c = 'red', s = 10, label = 'Outlier')
ax2.set(xlabel = 'PCA1', ylabel = 'PCA2', zlabel = 'Wine quality',
        title = 'Outlier detection on Wine data set\nusing Local Outlier Factor method')
plt.legend()

# The visualization of the effect of DBSCAN method
plt.figure(figsize = (5, 5))
ax3 = plt.axes(projection = '3d')
ax3.scatter3D(wine_DBSCAN['PCA1'][wine_DBSCAN['indicator'] == 0],
              wine_DBSCAN['PCA2'][wine_DBSCAN['indicator'] == 0],
              wine_DBSCAN['Quality'][wine_DBSCAN['indicator'] == 0],
              c = 'gray', s = 10, alpha = 0.25)
ax3.scatter3D(wine_DBSCAN['PCA1'][wine_DBSCAN['indicator'] == 1],
              wine_DBSCAN['PCA2'][wine_DBSCAN['indicator'] == 1],
              wine_DBSCAN['Quality'][wine_DBSCAN['indicator'] == 1],
              c = 'red', s = 10, label = 'Outlier')
ax3.scatter3D([], [], [], c = 'gray', s = 10, label = 'Inlier')
ax3.set(xlabel = 'PCA1', ylabel = 'PCA2', zlabel = 'Wine quality',
        title = 'Outlier detection on Wine data set\nusing DBSCAN method')
plt.legend()

## Part 2 - The exploration on the MNIST data set

Load the MNIST data set  
You first need to upload this data set to the following path on Kaggle:  
**'../input/mnist-dataset/mnist_784.csv'**  

Perform some initial analyses on MNIST data set

In [ ]:
# Load the MNIST data set and adjust the column name
# You first need to upload this data set to the following path:
# '../input/mnist-dataset/mnist_784.csv'
mnist = pd.read_csv('../input/mnist-dataset/mnist_784.csv', sep = ',')
mnist.rename(columns = {'class' : 'label'}, inplace = True)

# Fix the random seed
set_seed(seed = 200)

# Some initial analyses on the MNIST data set
dataset_size = len(mnist)
print(mnist.info()); display(mnist.describe())
display(mnist[['label']].join(pd.DataFrame({'count' : [1 for i in range(dataset_size)]})).\
    groupby('label').count())

# Some initial data preparation for the MNIST
mnist.iloc[:, :-1] = mnist.drop(['label'], 1)/255

I randomly take 120 samples from MNIST data set and visualize these digits.

In [ ]:
# Randomly take some samples from MNIST data set and visualize these digits
n_examples = 120
examples = np.random.permutation(mnist)[:n_examples, :]
features_exa = examples[:, :-1].reshape(n_examples, 28, 28)
labels_exa = examples[:, -1].astype('int')

columns = 15; rows = int(n_examples/columns)
figs, axes = plt.subplots(rows, columns, figsize = (columns, rows),
                          subplot_kw = {'xticks':[], 'yticks':[]},
                          gridspec_kw = dict(hspace = 0.1, wspace = 0.1))
plt.suptitle('Display of some handwritten digits in MNIST data set', fontsize = 15, y = 0.95)
for i, ax in enumerate(axes.flat):
    ax.imshow(features_exa[i], cmap = 'binary', interpolation = 'nearest')
    ax.text(0.05, 0.05, str(labels_exa[i]), transform = ax.transAxes, color = 'green')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)

Prepare the data used to train the model and split them into train, validation and test set

In [ ]:
# Prepare the data used to train the model
# Prepare the features and target of MNIST data set separately, and
# convert the data set to "tensorflow.dataset" type
mnist_onehot = OneHotEncoder()
mnist_onehot.fit(mnist[['label']].values)
labels = mnist_onehot.transform(mnist[['label']].values).A
features = tf.constant(mnist.loc[:, 'pixel1':'pixel784'].\
                       values.reshape(dataset_size, 28, 28, 1))
labels = tf.constant(labels)
mnist_tfds = tf.data.Dataset.from_tensor_slices((features, labels)).\
    shuffle(buffer_size = dataset_size).prefetch(buffer_size = dataset_size)

# Split the data set into train, validation and test set
train_size = round(dataset_size*0.85)
validation_size = round(dataset_size*0.075)
test_size = dataset_size - train_size - validation_size
mnist_train = mnist_tfds.take(train_size).batch(train_size)
mnist_validation = mnist_tfds.skip(train_size).take(validation_size).batch(validation_size)
mnist_test = mnist_tfds.skip(train_size + validation_size).batch(test_size)

Construct and train the Bayesian convolutional neural network (BCNN) model

In [ ]:
# Specify some model parameters
learning_rate = 0.002
num_epochs = 1000

# Define the loss function
# We still use the negative Evidence Lower Bound as the loss function
def negative_ELBO(label_true, label_pred):
    neg_log_likelihood = -tf.reduce_sum(label_pred.log_prob(label_true))
    kl = sum(mnist_model.losses)
    return neg_log_likelihood + kl/train_size

# Construct the Bayesian convolutional neural network model
mnist_model = tkeras.Sequential([
    tkeras.layers.Input(shape = (28, 28, 1)),
    tfp.layers.Convolution2DFlipout(filters = 8, kernel_size = (5, 5),
                                    padding = "SAME", activation = 'relu'),
    tkeras.layers.MaxPooling2D(pool_size = (2, 2), strides = (2, 2)),
    tfp.layers.Convolution2DFlipout(filters = 16, kernel_size = (5, 5), activation = 'relu'),
    tkeras.layers.MaxPooling2D(pool_size = (2, 2), strides = (2, 2)),
    tfp.layers.Convolution2DFlipout(filters = 128, kernel_size = (5, 5), activation = 'relu'),
    tkeras.layers.Flatten(),
    tkeras.layers.Dropout(0.5),
    tfp.layers.DenseFlipout(units = 96, activation = 'relu'),
    tkeras.layers.Dense(units = 10, activation = 'softmax'),
    tfp.layers.OneHotCategorical(event_size = 10,
                                 convert_to_tensor_fn = tfp.distributions.Distribution.mode)
])

# View the structure of the model
mnist_model.summary()

# Compile the constructed Bayesian convolutional neural network
# I use the Adam optimizer with learning rate being equal to 0.002 to minimize the
# loss function, and use the classification accuracy to evaluate the model accuracy.
mnist_model.compile(
    optimizer = tkeras.optimizers.Adam(learning_rate = learning_rate),
    loss = negative_ELBO,
    metrics = [tkeras.metrics.categorical_accuracy]
)

# Fit the constructed Bayesian convolutional neural network with data
mnist_fit = mnist_model.fit(x = mnist_train, epochs = num_epochs,
                            validation_data = mnist_validation)

Draw the trend of the loss and the MSE on the train and validation set during the training process respectively

In [ ]:
# Draw the trend of the loss and the classification accuracy on the
# train and validation set during the training process respectively

# Prepare the data
train_loss = mnist_fit.history['loss']
val_loss = mnist_fit.history['val_loss']
train_eva = mnist_fit.history['categorical_accuracy']
val_eva = mnist_fit.history['val_categorical_accuracy']
epochs = range(1, num_epochs + 1)

# The trend of loss on the train and validation set
fig1 = plt.figure(figsize = (5, 3)); ax1 = plt.axes()
ax1.plot(epochs, train_loss, 'b-.', label = 'Training loss')
ax1.plot(epochs, val_loss, 'g-.', label = 'Validation loss')
ax1.xaxis.set_major_locator(plt.MultipleLocator(250))
ax1.set(xlabel = 'Epoch', ylabel = 'Loss',
        title = 'Training and validation loss')
ax1.legend()

# The trend of classification accuracy on the train and validation set
fig2 = plt.figure(figsize = (5, 3)); ax2 = plt.axes()
ax2.plot(epochs, train_eva, 'b-', label = 'Training categorical accuracy')
ax2.plot(epochs, val_eva, 'g-', label = 'Validation categorical accuracy')
ax2.xaxis.set_major_locator(plt.MultipleLocator(250))
ax2.set(xlabel = 'Epoch', ylabel = 'Categorical accuracy',
        title = 'Training and validation categorical accuracy')
ax2.legend()

# Evaluate the trained model on both train set and test set respectively
print(mnist_model.evaluate(mnist_train, verbose = 0))
print(mnist_model.evaluate(mnist_test, verbose = 0))

Clear the model and re-train the model with the whole data set

In [ ]:
# Clear the model and re-train the model with the whole data set

# Appropriately reduce the data amount a little bit in order to avoid the problem of GPU memory overflow
# The principle and the implementation process of our method are always the same
real_size = dataset_size - 5000
mnist_real = mnist_tfds.take(real_size).batch(real_size)

# Re-define the loss function in order to update the weight for the
# KL divergence loss between the surrogate posterior and weight prior
def negative_ELBO(label_true, label_pred):
    neg_log_likelihood = -tf.reduce_sum(label_pred.log_prob(label_true))
    kl = sum(mnist_model.losses)
    return neg_log_likelihood + kl/real_size

# Clear the model
mnist_model = tkeras.Sequential([
    tkeras.layers.Input(shape = (28, 28, 1)),
    tfp.layers.Convolution2DFlipout(filters = 8, kernel_size = (5, 5),
                                    padding = "SAME", activation = 'relu'),
    tkeras.layers.MaxPooling2D(pool_size = (2, 2), strides = (2, 2)),
    tfp.layers.Convolution2DFlipout(filters = 16, kernel_size = (5, 5), activation = 'relu'),
    tkeras.layers.MaxPooling2D(pool_size = (2, 2), strides = (2, 2)),
    tfp.layers.Convolution2DFlipout(filters = 128, kernel_size = (5, 5), activation = 'relu'),
    tkeras.layers.Flatten(),
    tkeras.layers.Dropout(0.5),
    tfp.layers.DenseFlipout(units = 96, activation = 'relu'),
    tkeras.layers.Dense(units = 10, activation = 'softmax'),
    tfp.layers.OneHotCategorical(event_size = 10,
                                 convert_to_tensor_fn = tfp.distributions.Distribution.mode)
])

# Re-compile the model with the same settings
mnist_model.compile(
    optimizer = tkeras.optimizers.Adam(learning_rate = learning_rate),
    loss = negative_ELBO,
    metrics = [tkeras.metrics.categorical_accuracy]
)

# Re-fit the model with the whole data set
mnist_model.fit(x = mnist_real, epochs = num_epochs)

Quantify and plot all kinds of uncertainties of the predictions

In [ ]:
# Quantify all kinds of uncertainties of the predictions
N = 1000; records = np.zeros((N, real_size, 12))
features_real = features[:real_size]
labels_real = labels[:real_size]
labels_conv = np.where(labels_real.numpy())[1]
for i in range(N):
    records[i, :, 0:10] = mnist_model(features_real).mean().numpy()
    records[i, :, 10] = mnist_model(features_real).entropy().numpy()
    records[i, :, 11] = -np.sum(np.log(mnist_model(features_real).mean().numpy())*\
                                labels_real.numpy(), axis = 1)
epistemic = 0
for i in range(10):
    epistemic += np.var(records[:, :, i], axis = 0)
aleatoric = np.mean(records[:, :, 10], axis = 0)
misspecification = np.mean(records[:, :, 11], axis = 0)
P = 1/np.std(epistemic); Q = 1/np.std(aleatoric); R = 1/np.std(misspecification);
W = -(P*np.min(epistemic) + Q*np.min(aleatoric) + R*np.min(misspecification))
comp_uncer = P*epistemic + Q*aleatoric + R*misspecification + W

# Plot all kinds of uncertainties

# Plot the distribution of the epistemic uncertainty
plt.figure(figsize = (5, 3))
plt.hist(epistemic, color = 'blue', bins = 25,
         weights = np.ones(real_size)/real_size)
plt.xlabel('Epistemic uncertainty')
plt.ylabel('Frenquency')
plt.title('The distribution of the epistemic uncertainty')

# Plot the distribution of the aleatoric uncertainty
plt.figure(figsize = (5, 3))
plt.hist(aleatoric, color = 'green', bins = 25,
         weights = np.ones(real_size)/real_size)
plt.xlabel('Aleatoric uncertainty')
plt.ylabel('Frenquency')
plt.title('The distribution of the aleatoric uncertainty')

# Plot the distribution of the model misspecification uncertainty
plt.figure(figsize = (5, 3))
plt.hist(misspecification, color = 'red', bins = 25,
         weights = np.ones(real_size)/real_size)
plt.xlabel('Misspecification uncertainty')
plt.ylabel('Frenquency')
plt.title('The distribution of the misspecification uncertainty')

# Plot the distribution of the total prediction uncertainty
plt.figure(figsize = (5, 3))
plt.hist(comp_uncer, color = 'purple', bins = 25,
         weights = np.ones(real_size)/real_size)
plt.xlabel('Total prediction uncertainty')
plt.ylabel('Frenquency')
plt.title('The distribution of the total prediction uncertainty')
threshold = 13
plt.axvline(threshold, color = 'red')

Visualize the samples with high epistemic, aleatoric and model misspecification uncertainty respectievly

In [ ]:
# Determine a threshold about the uncertainty level
quantile = 0.985

# Visualize the samples with high epistemic uncertainty
location = (epistemic > np.quantile(epistemic, quantile))
index = np.where(location)[0]; amount = len(index)
columns = 15; rows = int(amount/columns) + 1 \
    if amount%columns != 0 else int(amount/columns)
figs, axes = plt.subplots(rows, columns, figsize = (columns, rows),
                          subplot_kw = {'xticks':[], 'yticks':[]},
                          gridspec_kw = dict(hspace = 0.4, wspace = 0.1))
plt.suptitle('The samples with high epistemic uncertainty', y = 0.89, fontsize = 15)
for i, ax in enumerate(axes.flat):
    if i < amount:
        ax.imshow(features_real.numpy()[index[i]], cmap = 'binary', interpolation = 'nearest')
        ax.text(0.05, 0.05, str(int(labels_conv[index[i]])),
                transform = ax.transAxes, color = 'green')
        ax.set_xlabel('{:.5f}'.format(epistemic[index[i]]), color = 'red')
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)
    else:
        ax.axis('off')

# Visualize the samples with high aleatoric uncertainty
location = (aleatoric > np.quantile(aleatoric, quantile))
index = np.where(location)[0]; amount = len(index)
columns = 15; rows = int(amount/columns) + 1 \
    if amount%columns != 0 else int(amount/columns)
figs, axes = plt.subplots(rows, columns, figsize = (columns, rows),
                          subplot_kw = {'xticks':[], 'yticks':[]},
                          gridspec_kw = dict(hspace = 0.4, wspace = 0.1))
plt.suptitle('The samples with high aleatoric uncertainty', y = 0.89, fontsize = 15)
for i, ax in enumerate(axes.flat):
    if i < amount:
        ax.imshow(features_real.numpy()[index[i]], cmap = 'binary', interpolation = 'nearest')
        ax.text(0.05, 0.05, str(int(labels_conv[index[i]])),
                transform = ax.transAxes, color = 'green')
        ax.set_xlabel('{:.5f}'.format(aleatoric[index[i]]), color = 'red')
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)
    else:
        ax.axis('off')

# Visualize the samples with high misspecification uncertainty
location = (misspecification > np.quantile(misspecification, quantile))
index = np.where(location)[0]; amount = len(index)
columns = 15; rows = int(amount/columns) + 1 \
    if amount%columns != 0 else int(amount/columns)
figs, axes = plt.subplots(rows, columns, figsize = (columns, rows),
                          subplot_kw = {'xticks':[], 'yticks':[]},
                          gridspec_kw = dict(hspace = 0.4, wspace = 0.1))
plt.suptitle('The samples with high misspecification uncertainty', y = 0.89, fontsize = 15)
for i, ax in enumerate(axes.flat):
    if i < amount:
        ax.imshow(features_real.numpy()[index[i]], cmap = 'binary', interpolation = 'nearest')
        ax.text(0.05, 0.05, str(int(labels_conv[index[i]])),
                transform = ax.transAxes, color = 'green')
        ax.set_xlabel('{:.4f}'.format(misspecification[index[i]]), color = 'red')
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)
    else:
        ax.axis('off')

Visualize the result of outlier detection on MNIST data set using BCNN

In [ ]:
# Record the information about the index of outliers
location = (comp_uncer > threshold)
index = np.where(location)[0]
n_outliers = len(index)
print(np.mean(location)); print(n_outliers)

# Visualize the result of outlier detection on MNIST data set using BCNN
columns = 15; rows = int(n_outliers/columns) + 1 \
    if n_outliers%columns != 0 else int(n_outliers/columns)
figs, axes = plt.subplots(rows, columns, figsize = (columns, rows),
                          subplot_kw = {'xticks':[], 'yticks':[]},
                          gridspec_kw = dict(hspace = 0.4, wspace = 0.1))
plt.suptitle('Outlier detection on MNIST data set using BCNN', y = 0.89, fontsize = 15)
for i, ax in enumerate(axes.flat):
    if i < n_outliers:
        ax.imshow(features_real.numpy()[index[i]], cmap = 'binary', interpolation = 'nearest')
        ax.text(0.05, 0.05, str(int(labels_conv[index[i]])),
                transform = ax.transAxes, color = 'green')
        ax.set_xlabel('{:.2f}'.format(comp_uncer[index[i]]), color = 'red')
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)
    else:
        ax.axis('off')

Perform outlier detection on MNIST data set using LOF and DBSCAN method respectively, and visualize the results of these two methods

In [ ]:
# Outlier detection on MNIST data set using Local Outlier Factor method
Lof = LocalOutlierFactor(); offset = -1.5
Lof.fit(mnist)
Lof_fac = Lof.negative_outlier_factor_
Lof_pre = (Lof_fac < offset).astype('int')
index_out = (Lof_pre == 1)
n_outliers = sum(index_out)
outliers_feature = mnist[index_out].drop('label', 1).\
                   values.reshape(n_outliers, 28, 28)
outliers_label = mnist[index_out]['label'].values

# Visualize the result of outlier detection on MNIST data set using LOF method
columns = 15; rows = int(n_outliers/columns) + 1 \
    if n_outliers%columns != 0 else int(n_outliers/columns)
figs, axes = plt.subplots(rows, columns, figsize = (columns, rows),
                          subplot_kw = {'xticks':[], 'yticks':[]},
                          gridspec_kw = dict(hspace = 0.4, wspace = 0.1))
plt.suptitle('Outlier detection on MNIST data set using LOF method', y = 0.9, fontsize = 15)
for i, ax in enumerate(axes.flat):
    if i < n_outliers:
        ax.imshow(outliers_feature[i], cmap = 'binary', interpolation = 'nearest')
        ax.text(0.05, 0.05, str(outliers_label[i]),
                transform = ax.transAxes, color = 'green')
        ax.set_xlabel('{:.2f}'.format(-Lof_fac[np.where(index_out)[0][i]]), color = 'red')
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)
    else:
        ax.axis('off')

# Outlier detection on MNIST data set using DBSCAN method
DBSCAN_model = DBSCAN(eps = 7.2, min_samples = 8)
DBSCAN_fit = DBSCAN_model.fit(mnist)
DBSCAN_pre = DBSCAN_fit.labels_
index_noise = (DBSCAN_pre == -1); n_noise = sum(index_noise)
noise_feature = mnist[index_noise].drop('label', 1).\
                values.reshape(n_noise, 28, 28)
noise_label = mnist[index_noise]['label'].values

# Visualize the result of outlier detection on MNIST data set using DBSCAN method
columns = 15; rows = int(n_noise/columns) + 1 \
    if n_noise%columns != 0 else int(n_noise/columns)
figs, axes = plt.subplots(rows, columns, figsize = (columns, rows),
                          subplot_kw = {'xticks':[], 'yticks':[]},
                          gridspec_kw = dict(hspace = 0.1, wspace = 0.1))
plt.suptitle('Outlier detection on MNIST data set using DBSCAN method', y = 0.9, fontsize = 15)
for i, ax in enumerate(axes.flat):
    if i < n_noise:
        ax.imshow(noise_feature[i], cmap = 'binary', interpolation = 'nearest')
        ax.text(0.05, 0.05, str(noise_label[i]),
                transform = ax.transAxes, color = 'green')
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)
    else:
        ax.axis('off')

## Part 3 - The exploration on the Taxi data set

Load the Taxi data set  
You first need to upload this data set to the following path on Kaggle:  
**'../input/taxi-dataset/taxi.csv'**  

Perform some initial analyses on Taxi data set and remove the seasonal variation of Taxi data set

In [ ]:
# Load the Taxi data set and use the 'timestamp' column as index
# You first need to upload this data set to the following path:
# '../input/taxi-dataset/taxi.csv'
taxi = pd.read_csv('../input/taxi-dataset/taxi.csv',
                   index_col = 'timestamp', parse_dates = True)
taxi.index.freq = '30T'

# Fix the random seed
set_seed(seed = 120)

# Some initial analyses on Taxi data set
print(taxi.info()); display(taxi.describe())
taxi.plot(figsize = (7.2, 3.6), xlabel = 'Date', ylabel = 'Taxi passengers\' count',
          title = 'Counts of taxi passengers in New-York city').legend_.remove()

# Remove the seasonal variation of Taxi data set
period = 336
myfilter = np.hstack([1/(2*period), np.array([1/period for i in range(period - 1)]),
                      1/(2*period)])
taxi_decom = seasonal_decompose(taxi, filt = myfilter, period = period,
                                extrapolate_trend = 'freq')
taxi_DeSeason = taxi_decom.observed - taxi_decom.seasonal
display(taxi_DeSeason.describe())

# Visualize the Taxi data set after removing seasonal variation
plt.figure(figsize = (7.2, 3.6))
plt.plot(taxi_DeSeason.index, taxi_DeSeason.values)
plt.xlabel('Date'); plt.ylabel('Taxi passengers\' count')
plt.title('Counts of taxi passengers after removing seasonal variation')

Prepare the data used to train the model and split them into train, validation and test set

In [ ]:
# Prepare the data used to train the model
# Prepare the features and target of Taxi data set separately, and
# convert the data set to "tensorflow.dataset" type
time = (taxi.index.values - np.datetime64('2014-07-01T00:00:00'))/np.timedelta64(1, '30m')
taxi_data = np.vstack([time, taxi_DeSeason.values]).T
dataset_size = len(taxi_data) - 1
length = 24
features = np.zeros((dataset_size, length, 2))
for i in range(dataset_size):
    if i < length:
        features[i] = np.vstack([taxi_data[[0 for k in range(length - i - 1)]],
                                 taxi_data[[k for k in range(i + 1)]]])
    else:
        features[i] = taxi_data[(i - length + 1):(i + 1)]
features = tf.constant(features)
labels = tf.constant(taxi_DeSeason.values[1:])
taxi_tfds = tf.data.Dataset.from_tensor_slices((features, labels)).\
    map(lambda x, y: (x, tf.cast(y, tf.float64))).\
    prefetch(buffer_size = dataset_size)

# Split the data set into train, validation and test set
train_size = round(dataset_size*0.8)
validation_size = round(dataset_size*0.1)
test_size = dataset_size - train_size - validation_size
taxi_tfds_veri = taxi_tfds.shuffle(buffer_size = dataset_size)
taxi_train = taxi_tfds_veri.take(train_size).batch(train_size)
taxi_validation = taxi_tfds_veri.skip(train_size).take(validation_size).batch(validation_size)
taxi_test = taxi_tfds_veri.skip(train_size + validation_size).batch(test_size)

Construct and train the Bayesian LSTM neural network (BLSTMNN) model

In [ ]:
# Specify some model parameters
num_epochs = 5000
learning_rate = 0.5

# Define the loss function
# We still use the negative Evidence Lower Bound (-ELBO) as the loss function.
def negative_ELBO(label_true, label_pred):
    neg_log_likelihood = -tf.reduce_sum(label_pred.log_prob(label_true))
    kl = sum(taxi_model.losses)/train_size
    return neg_log_likelihood + kl

# Construct the Bayesian LSTM neural network (BLSTMNN) model
BLSTM_layer1 = ed2.layers.LSTMCellFlipout(8, activation = 'sigmoid')
BLSTM_layer2 = ed2.layers.LSTMCellFlipout(16, activation = 'sigmoid')
taxi_model = tkeras.Sequential([
    tkeras.layers.Input(shape = (length, 2)),
    tkeras.layers.BatchNormalization(),
    tkeras.layers.RNN(cell = BLSTM_layer1, return_sequences = True),
    tkeras.layers.RNN(cell = BLSTM_layer2),
    tkeras.layers.Dense(units = 2),
    tfp.layers.IndependentNormal(1)
])

# View the structure of the model
taxi_model.summary()

# Compile the constructed Bayesian LSTM neural network
# I use the RMSprop optimizer with learning rate being equal to 0.5 to
# minimize the loss function, and use Mean Square Error (MSE) as the metric
# to evaluate the accuracy of the model.
taxi_model.compile(
    optimizer = tkeras.optimizers.RMSprop(learning_rate = learning_rate),
    loss = negative_ELBO,
    metrics = [tkeras.metrics.mean_squared_error]
)

# Fit the constructed Bayesian LSTM neural network with data
taxi_fit = taxi_model.fit(x = taxi_train, epochs = num_epochs,
                          validation_data = taxi_validation)

Draw the trend of the loss and the MSE on the train and validation set during the training process respectively

In [ ]:
# Draw the trend of the loss and the MSE on the train and
# validation set during the training process respectively

# Prepare the data
train_loss = np.array(taxi_fit.history['loss'])
val_loss = np.array(taxi_fit.history['val_loss'])
train_eva = np.array(taxi_fit.history['mean_squared_error'])
val_eva = np.array(taxi_fit.history['val_mean_squared_error'])
inits = 10
epochs = np.array(range(inits, len(train_loss)))

# The trend of loss on the train and validation set
fig1 = plt.figure(figsize = (5, 3)); ax1 = plt.axes()
ax1.plot(epochs + 1, train_loss[epochs], 'b-.', label = 'Training loss')
ax1.plot(epochs + 1, val_loss[epochs], 'g-.', label = 'Validation loss')
ax1.set(xlabel = 'Epoch', ylabel = 'Loss',
        title = 'Training and validation loss')
ax1.legend()

# The trend of the metric (MSE) on the train and validation set
fig2 = plt.figure(figsize = (5, 3)); ax2 = plt.axes()
ax2.plot(epochs + 1, val_eva[epochs], 'g-', label = 'Validation MSE')
ax2.plot(epochs + 1, train_eva[epochs], 'b-', label = 'Training MSE')
ax2.set(xlabel = 'Epoch', ylabel = 'Mean squared error',
        title = 'Training and validation MSE')
ax2.legend()

# Evaluate the trained model on both train set and test set respectively
print(taxi_model.evaluate(taxi_train, verbose = 0))
print(taxi_model.evaluate(taxi_test, verbose = 0))

I take 10 samples from the test set. Then, I construct and visualize the 95% confidence intervals for the predictions of these samples.

In [ ]:
# Take 10 samples from the test set
num_exa = 10
features_exa, targets_exa = list(taxi_test.unbatch().batch(num_exa))[0]
features_exa = features_exa.numpy()
targets_exa = targets_exa.numpy()

# Compare the prediction means with the true labels
examples_mean = taxi_model(features_exa).mean().numpy()
examples_std = taxi_model(features_exa).stddev().numpy()

# Construct and visualize the 95% confidence intervals for the predictions
plt.figure(figsize = (10, 5))
index = range(1, num_exa + 1)
plt.scatter(index, examples_mean, color = 'red', label = 'Prediction mean')
plt.scatter(index, targets_exa, color = 'green', label = 'True value')
plt.scatter(index, examples_mean + 1.96*examples_std,
            color = 'blue', marker = '_', label = 'Confidence bound')
plt.scatter(index, examples_mean - 1.96*examples_std,
            color = 'blue', marker = '_')
plt.xlabel('Sample index'); plt.ylabel('Taxi passengers\' count')
plt.title('The 95% confidence intervals for the predictions of ten samples')
plt.legend()

Clear the model and re-train the model with the whole data set

In [ ]:
# Clear the model and re-train the model with the whole data set

# Re-define the loss function in order to update the weight for the
# KL divergence loss between the surrogate posterior and weight prior
def negative_ELBO(label_true, label_pred):
    neg_log_likelihood = -tf.reduce_sum(label_pred.log_prob(label_true))
    kl = sum(taxi_model.losses)
    return neg_log_likelihood + kl/dataset_size

# Clear the model
BLSTM_layer1 = ed2.layers.LSTMCellFlipout(8, activation = 'sigmoid')
BLSTM_layer2 = ed2.layers.LSTMCellFlipout(16, activation = 'sigmoid')
taxi_model = tkeras.Sequential([
    tkeras.layers.Input(shape = (length, 2)),
    tkeras.layers.BatchNormalization(),
    tkeras.layers.RNN(cell = BLSTM_layer1, return_sequences = True),
    tkeras.layers.RNN(cell = BLSTM_layer2),
    tkeras.layers.Dense(units = 2),
    tfp.layers.IndependentNormal(1)
])

# Re-compile the model with the same settings
taxi_model.compile(
    optimizer = tkeras.optimizers.RMSprop(learning_rate = learning_rate),
    loss = negative_ELBO,
    metrics = [tkeras.metrics.mean_squared_error]
)

# Re-fit the model with the whole data set
taxi_model.fit(x = taxi_tfds.batch(dataset_size), epochs = num_epochs)

Quantify and plot all kinds of uncertainties of the predictions

In [ ]:
# Quantify all kinds of uncertainties of the predictions
N = 1000; records = np.zeros((N, dataset_size, 3))
for i in range(N):
    records[i, :, 0] = taxi_model(features).mean().numpy()[:, 0]
    records[i, :, 1] = taxi_model(features).variance().numpy()[:, 0]
    records[i, :, 2] = (taxi_model(features).mean().numpy()[:, 0] - labels.numpy())**2
epistemic = np.var(records[:, :, 0], axis = 0)
aleatoric = np.mean(records[:, :, 1], axis = 0)
misspecification = np.mean(records[:, :, 2], axis = 0)
P = 0; Q = 1/np.std(aleatoric); R = 6/np.std(misspecification)
W = -(Q*np.min(aleatoric) + R*np.min(misspecification))
comp_uncer = P*epistemic + Q*aleatoric + R*misspecification + W

# Plot all kinds of uncertainties

# Plot the distribution of the epistemic uncertainty
plt.figure(figsize = (5, 3))
plt.hist(epistemic, color = 'blue', bins = 25,
         weights = np.ones(dataset_size)/dataset_size)
plt.ticklabel_format(style = 'sci', scilimits = (0, 4), axis = 'x')
plt.xlabel('Epistemic uncertainty')
plt.ylabel('Frenquency')
plt.title('The distribution of the epistemic uncertainty')

# Plot the distribution of the aleatoric uncertainty
plt.figure(figsize = (5, 3))
plt.hist(aleatoric, color = 'green', bins = 25,
         weights = np.ones(dataset_size)/dataset_size)
plt.xlim(pow(10, 6))
plt.xlabel('Aleatoric uncertainty')
plt.ylabel('Frenquency')
plt.title('The distribution of the aleatoric uncertainty')

# Plot the distribution of the model misspecification uncertainty
plt.figure(figsize = (5, 3))
plt.hist(misspecification, color = 'red', bins = 25,
         weights = np.ones(dataset_size)/dataset_size)
plt.xlabel('Misspecification uncertainty')
plt.ylabel('Frenquency')
plt.title('The distribution of the misspecification uncertainty')

# Plot the distribution of the total prediction uncertainty
plt.figure(figsize = (5, 3))
plt.hist(comp_uncer, color = 'purple', bins = 25,
         weights = np.ones(dataset_size)/dataset_size)
plt.xlabel('Total prediction uncertainty')
plt.ylabel('Frenquency')
plt.title('The distribution of the total prediction uncertainty')
threshold = np.quantile(comp_uncer, 0.975)
plt.axvline(threshold, color = 'red')

# Adjust the vectors that contain all kinds of uncertainties
# in order to facilitate performing outlier detection
epistemic = np.hstack([0, epistemic])
aleatoric = np.hstack([0, aleatoric])
misspecification = np.hstack([0, misspecification])
comp_uncer = np.hstack([0, comp_uncer])

Visualize the observations with high epistemic, aleatoric and model misspecification uncertainty respectievly

In [ ]:
# Determine a threshold about the uncertainty level
quantile = 0.975

# Visualize the observations with high epistemic uncertainty
index = (epistemic > np.quantile(epistemic, quantile))
high_epi = taxi_DeSeason[index]
norm_epi = taxi_DeSeason[(1 - index).astype('bool')]
fig = plt.figure(figsize = (7.2, 3.6)); ax = plt.axes()
ax.scatter(norm_epi.index, norm_epi.values, label = 'Normal')
ax.scatter(high_epi.index, high_epi.values, color = 'red',
           s = plt.rcParams['lines.markersize']**2 + 3*epistemic[index]/pow(10, 5))
ax.scatter([], [], color = 'red', label = 'High uncertainty')
ax.set(xlabel = 'Time', ylabel = 'Taxi passengers\' count',
       title = 'The observations with high epistemic uncertainty')
plt.legend()

# Visualize the observations with high aleatoric uncertainty
index = (aleatoric > np.quantile(aleatoric, quantile))
high_alea = taxi_DeSeason[index]
norm_alea = taxi_DeSeason[(1 - index).astype('bool')]
fig = plt.figure(figsize = (7.2, 3.6)); ax = plt.axes()
ax.scatter(norm_alea.index, norm_alea.values, label = 'Normal')
ax.scatter(high_alea.index, high_alea.values, color = 'red',
           s = plt.rcParams['lines.markersize']**2 + 4*aleatoric[index]/pow(10, 6))
ax.scatter([], [], color = 'red', label = 'High uncertainty')
ax.set(xlabel = 'Time', ylabel = 'Taxi passengers\' count',
       title = 'The observations with high aleatoric uncertainty')
plt.legend()

# Visualize the observations with high model misspecification uncertainty
index = (misspecification > np.quantile(misspecification, quantile))
high_mis = taxi_DeSeason[index]
norm_mis = taxi_DeSeason[(1 - index).astype('bool')]
fig = plt.figure(figsize = (7.2, 3.6)); ax = plt.axes()
ax.scatter(norm_mis.index, norm_mis.values, label = 'Normal')
ax.scatter(high_mis.index, high_mis.values, color = 'red',
           s = plt.rcParams['lines.markersize']**2 + 3*misspecification[index]/pow(10, 8))
ax.scatter([], [], color = 'red', label = 'High uncertainty')
ax.set(xlabel = 'Time', ylabel = 'Taxi passengers\' count',
       title = 'The observations with high misspecification uncertainty')
plt.legend()

Visualize the result of outlier detection on Taxi data set using BLSTMNN

In [ ]:
# Record the information about the index of outliers
threshold = np.quantile(comp_uncer, 0.975)
index = (comp_uncer > threshold)
n_outliers = np.sum(index)
print(np.mean(index)); print(n_outliers)

# Visualize the result of outlier detection on Taxi data set
# using Bayesian LSTM neural network
outliers = taxi_DeSeason[index]
inliers = taxi_DeSeason[(1 - index).astype('bool')]
fig = plt.figure(figsize = (7.2, 3.6)); ax = plt.axes()
ax.scatter(inliers.index, inliers.values, label = 'Inliers')
ax.scatter(outliers.index, outliers.values, color = 'red',
           s = plt.rcParams['lines.markersize']**2 + comp_uncer[index]/4)
ax.scatter([], [], color = 'red', label = 'Outliers')
ax.set(xlabel = 'Time', ylabel = 'Taxi passengers\' count',
       title = 'Counts of taxi passengers in New-York after removing seasonality')
plt.legend()

# Visualize the duration of the periods experiencing extremely high and extremely low
# taxi passenger numbers per day in New-York city respectively, and compare the dates
# of these extreme periods with the dates of special events happened in New York

# Prepare the data that we need
taxi_count = taxi.copy(); middle = 15000
taxi_count['value'] = taxi_DeSeason.values
large = []; small = []
for i in range(dataset_size + 1):
    x = 1 if (index[i] and taxi_DeSeason.values[i] > middle) else 0
    large.append(x)
    x = 1 if (index[i] and taxi_DeSeason.values[i] < middle) else 0
    small.append(x)
taxi_count['count_large'] = large
taxi_count['count_small'] = small
taxi_count = taxi_count.resample('D').sum().\
    apply(lambda x : 0.5*x).rename(columns = {'value' : 'busyness'})

# Visualize the duration of the extreme periods per day, and compare the
# dates of these extreme periods with the dates of special events in New-York
fig = plt.figure(figsize = (7.2, 3.6)); ax = plt.axes()
ax.plot(taxi_count.index, taxi_count['count_large'],
        color = 'red', label = 'Extremely high period')
ax.plot(taxi_count.index, taxi_count['count_small'],
        color = 'blue', label = 'Extremely low period')
events = ['Independence Day', 'School opening day', 'New-York marathon',
          'Thanksgiving Day', 'Christmas', 'New Year\'s Day', 'A snow storm']
dates = [pd.Timestamp('2014-7-4'), pd.Timestamp('2014-9-1'),
         pd.Timestamp('2014-11-2'), pd.Timestamp('2014-11-27'),
         pd.Timestamp('2014-12-25'), pd.Timestamp('2015-1-1'),
         pd.Timestamp('2015-1-26')]
colors = ['green', 'purple', 'yellow', 'brown', 'fuchsia', 'orange', 'cyan']
for event, date, color in zip(events, dates, colors):
    ax.axvline(date, label = event, color = color, linestyle = '--')
ax.set(xlabel = 'Date', ylabel = 'Period length (Hour)',
       title = 'Duration of extreme periods per day in New-York city')
plt.legend(bbox_to_anchor = (1.03, 0.5), loc = 6, borderaxespad = 0)

Perform outlier detection on Taxi data set using LOF and DBSCAN method respectively, and visualize the results of these two methods

In [ ]:
# Prepare data for the LOF method and DBSCAN method
dataset_size = len(taxi_DeSeason)
taxi_detect = pd.DataFrame(
    np.array([range(dataset_size), taxi_DeSeason.values]).T,
    columns = ['time', 'counts']
)

# Outlier detection on Taxi data set using Local Outlier Factor method
Lof = LocalOutlierFactor(); offset_ = -1.45
Lof.fit(taxi_detect)
Lof_fac = Lof.negative_outlier_factor_
Lof_pre = -(Lof_fac < offset_).astype('int')
index_out = (Lof_pre == -1); index_in = (1 - index_out).astype('bool')
outliers = taxi_DeSeason[index_out]; inliers = taxi_DeSeason[index_in]

# Visualize the result of outlier detection on Taxi data set using LOF method
fig = plt.figure(figsize = (7.2, 3.6)); ax = plt.axes()
ax.scatter(inliers.index, inliers.values, c = Lof_pre[index_in],
           cmap = plt.cm.Dark2, label = 'Inliers')
ax.scatter(outliers.index, outliers.values, color = 'red',
           s = plt.rcParams['lines.markersize']**2 - 6*Lof_fac[index_out])
ax.scatter([], [], color = 'red', label = 'Outliers')
ax.set(xlabel = 'Time', ylabel = 'Taxi passengers\' count',
       title = 'Counts of taxi passengers in New-York after removing seasonality')
plt.legend()

# Outlier detection on Taxi data set using DBSCAN method
DBSCAN_model = DBSCAN(eps = 380, min_samples = 20)
DBSCAN_fit = DBSCAN_model.fit(taxi_detect)
DBSCAN_pre = DBSCAN_fit.labels_
index_noise = (DBSCAN_pre == -1); index_in = (1 - index_noise).astype('bool')
noise = taxi_DeSeason[index_noise]; inliers = taxi_DeSeason[index_in]

# Visualize the result of outlier detection on Taxi data set using DBSCAN method
fig = plt.figure(figsize = (7.2, 3.6)); ax = plt.axes()
ax.scatter(inliers.index, inliers.values, c = DBSCAN_pre[index_in],
           cmap = plt.cm.Dark2, label = 'Inliers')
ax.scatter(noise.index, noise.values, color = 'red', label = 'Outliers')
ax.set(xlabel = 'Time', ylabel = 'Taxi passengers\' count',
       title = 'Counts of taxi passengers in New-York after removing seasonality')
plt.legend()